#  Document Extraction MVP — Interactive Search (Variant E)

This notebook provides:
- Interactive search interface
- JSON rendering
- Constitution / Mathematics / Utility filters
- SQLite database search UI


In [ ]:

!pip install ipywidgets


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output, JSON, HTML
from load_db import DocumentDatabase

# connect database
db = DocumentDatabase("data/documents.db")

print("Database connected")


INFO:load_db:Connected to database: data/documents.db
INFO:load_db:Database tables created successfully


Database connected


In [ ]:
# ------------- UI ------------------

doc_type = widgets.Dropdown(
    options=['constitution','mathematics','utility'],
    value='constitution',
    description='Document:'
)

search_box = widgets.Text(
    placeholder='Enter search query...',
    description='Search:'
)

search_btn = widgets.Button(
    description='Search',
    button_style='primary'
)

clear_btn = widgets.Button(
    description='Clear',
    button_style='warning'
)

output = widgets.Output()
status = widgets.HTML("Ready to search...")

# ---------- SEARCH FUNCTION -----------

def perform_search(b):

    with output:
        clear_output()

        query = search_box.value.strip()
        dtype = doc_type.value

        if not query:
            print("Enter search query")
            return

        status.value = f"Searching {dtype} for '{query}'..."

        try:

            # ---------- CONSTITUTION SEARCH ----------
            if dtype == "constitution":

                results = []

                # existing helpers
                results.extend(db.search_article(query))

                # extra fields search
                db.cursor.execute("""
                SELECT * FROM constitution
                WHERE article_title LIKE ?
                OR chapter LIKE ?
                OR raw_text LIKE ?
                """, (f"%{query}%", f"%{query}%", f"%{query}%"))

                results.extend([dict(r) for r in db.cursor.fetchall()])


            # ---------- MATHEMATICS SEARCH ----------
            elif dtype == "mathematics":

                results = db.get_theorems(query)

                db.cursor.execute("""
                SELECT * FROM mathematics
                WHERE section_name LIKE ?
                OR raw_text LIKE ?
                """, (f"%{query}%", f"%{query}%"))

                results.extend([dict(r) for r in db.cursor.fetchall()])


            # ---------- UTILITY SEARCH ----------
            elif dtype == "utility":

                results = db.filter_location(query)

                db.cursor.execute("""
                SELECT * FROM utility
                WHERE entity_id LIKE ?
                OR date LIKE ?
                OR unit LIKE ?
                OR raw_text LIKE ?
                """, (f"%{query}%", f"%{query}%", f"%{query}%", f"%{query}%"))

                results.extend([dict(r) for r in db.cursor.fetchall()])


            # ---------- REMOVE DUPLICATES ----------
            seen = set()
            unique_results = []

            for r in results:
                rid = r.get("id")
                if rid not in seen:
                    seen.add(rid)
                    unique_results.append(r)

            # ---------- DISPLAY ----------
            if unique_results:
                print(f"Found {len(unique_results)} result(s)\n")
                for r in unique_results:
                    display(JSON(r))
            else:
                print("No results found")

            status.value = "Search complete"

        except Exception as e:
            print("ERROR:", e)
            status.value = "Error occurred"
# ------------ BUTTON ACTIONS ----------

search_btn.on_click(perform_search)

def clear_results(b):
    search_box.value=""
    with output:
        clear_output()

clear_btn.on_click(clear_results)

# -----------LAYOUT ----------------

ui = widgets.VBox([
    widgets.HTML("<h2>🔍 Document Search Interface</h2>"),
    doc_type,
    search_box,
    widgets.HBox([search_btn, clear_btn]),
    status,
    output
])

display(ui)
